# Imports and Configurations

In [4]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from tqdm import tqdm
from util.StringObjectParser import StringObjectParser
import scipy.stats as stats


ModuleNotFoundError: No module named 'util'

In [ ]:
sns.set_theme(style="whitegrid")

# Load Data

## NL to SQL Performance

In [ ]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")
data_directory = current_directory.replace("/src", "/nl_sql_results")

xlsx_files = [file for file in os.listdir(data_directory) if file.endswith(".xlsx")]
dataframes = []
for xf in xlsx_files:
    df = pd.read_excel(os.path.join(data_directory, xf))
    xf_bits = xf.split("-")
    df["subsetting_method"] = xf_bits[1]
    df["benchmark"] = xf_bits[2]
    df["comments"] = xf_bits[4].replace(".xlsx", "")
    dataframes.append(df)
nlsql_performance_df = pd.concat(dataframes, ignore_index=True)

## Subsetting Performance

In [ ]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")
data_directory = current_directory.replace("/src", "/subsetting_results")

xlsx_files = [file for file in os.listdir(data_directory) if file.endswith('.xlsx')]
dataframes = []
for xf in xlsx_files:
    df = pd.read_excel(os.path.join(data_directory, xf))
    df["subsetting_method"] = xf.split("-")[1]
    df["benchmark"] = xf.split("-")[2]
    df["comments"] = xf.split("-")[4].replace(".xlsx", "")
    dataframes.append(df)
subset_performance_df = pd.concat(dataframes, ignore_index=True)
for column in subset_performance_df.columns:
    processed_objs = []
    for val in subset_performance_df[column]:
        obj = StringObjectParser.string_to_python_object(val, use_eval=True)
        if type(obj) == str and obj[0] == "{":
            obj = set([v.strip() for v in obj.replace("{", "").replace("}", "").split(",")])
        processed_objs.append(obj)
    subset_performance_df[column] = processed_objs
        # subset_performance_df[column] = subset_performance_df[column].apply(
        #     lambda x: StringObjectParser.string_to_python_object(x, use_eval=True) if StringObjectParser.check_valid_container(x) or x == "set()" else x
        # )
    

## Schema Stats

In [ ]:
current_directory = f"{os.getcwd()}/benchmark_schema_stats"
data_directory = current_directory.replace("/src", "/subsetting_results")

json_files = [file for file in os.listdir(data_directory) if file.endswith('_stats.json')]
df_dict = {"benchmark": [], "database": [], "table_count": [], "column_count": []}
for filename in json_files:
    fn_split = filename.split("_")
    stat_dict = json.loads(open(f"{data_directory}/{filename}", "r").read())
    for k in stat_dict:
        df_dict["benchmark"].append(fn_split[0])
        df_dict["database"].append(k)
        df_dict["table_count"].append(stat_dict[k]["table_count"])
        df_dict["column_count"].append(stat_dict[k]["column_count"])
schema_stat_df = pd.DataFrame(df_dict).query("benchmark!='abstract'")
for column in schema_stat_df.columns:
    schema_stat_df[column] = schema_stat_df[column].apply(
        lambda x: StringObjectParser.string_to_python_object(x, use_eval=True) if StringObjectParser.check_valid_container(x) or x == "set()" else x
    )

# Process Data

## Model Filter

In [ ]:
comment_model_filter = {
    "CodeS": "lambda1_sic_merged",
    "DINSQL": "gpt41",
    "chess": "gpt4o",
    "crush4sql": "lambda1",
    "dtssql": "lambda1",
    "rslsql": "gpt41",
    "tasql": "gpt41",
    # "skalpel": "vector_qdecomp_525th",
    "skalpel": "do not eval",
    "skalpeltasql": "gpt41nano-vectorsort",
    "perfect_subsetter": "oracle",
    "perfect_table_subsetter": "oracle",
    "nosubset": "fullschema"
}

## Display Data Maps

In [ ]:
subsetting_display_names = {
    "crush4sql": "Crush",
    "rslsql": "RSLSQL",
    "tasql": "TASQL",
    "CodeS": "CodeS",
    "dtssql": "DTSSQL",
    "chess": "CHESS",
    "DINSQL": "DINSQL",
    "skalpel": "Skalpel",
    "skalpeltasql": "Skalpel+TASQL",
    "perfect_subsetter": "Perfect",
    "perfect_table_subsetter": "Perfect-Table",
    "nosubset": "FullSchema"
}

In [ ]:
subsetting_classifications = {
    "crush4sql": "Hyb",
    "rslsql": "LLM",
    "tasql": "LLM",
    "CodeS": "ML",
    "dtssql": "LLM",
    "chess": "Hyb",
    "DINSQL": "LLM",
    "Skalpel": "Hyb",
    "Skalpel+TASQL": "Hyb",
    "perfect_subsetter": "Oracle",
    "perfect_table_subsetter": "Oracle",
    "nosubset": "Full"
}

In [ ]:
subsetting_display_order = {
    "Crush": 80,
    "RSLSQL": 40,
    "TASQL": 50,
    "CodeS": 90,
    "DTSSQL": 100,
    "CHESS": 70,
    "DINSQL": 60,
    "Skalpel": 30,
    "Skalpel+TASQL": 35,
    "Perfect": 10,
    "Perfect-Table": 20,
    "FullSchema": 1000
}

## Process Performance Data

In [ ]:
nlsql_performance_df["Subsetting Method"] = nlsql_performance_df.subsetting_method.apply(lambda x: subsetting_display_names[x])
subset_performance_df["Subsetting Method"] = subset_performance_df.subsetting_method.apply(lambda x: subsetting_display_names[x])

In [ ]:
nlsql_performance_df["eval_model"] = nlsql_performance_df.apply(
    lambda row: row.comments == comment_model_filter[row.subsetting_method], 
    axis=1
    )
subset_performance_df["eval_model"] = subset_performance_df.apply(
    lambda row: row.comments == comment_model_filter[row.subsetting_method], 
    axis=1
    )
nlsql_performance_df["nl_sql_model"] = nlsql_performance_df.nl_sql_model.fillna("openai/gpt-oss-120b")

In [ ]:
nlsql_performance_df["method_order"] = nlsql_performance_df["Subsetting Method"].apply(
    lambda x: subsetting_display_order[x]
    )
nlsql_performance_df["has_generated_query"] = nlsql_performance_df.generated_query.fillna("doh").apply(lambda x: x != "doh" and x != 'Token limited exceeded')

In [ ]:
subset_performance_df["perfect_recall"] = subset_performance_df.total_recall.apply(lambda x: 1 if x == 1.0 else 0)
subset_performance_df["empty_subset"] = subset_performance_df.apply(
    lambda row: len(row.correct_tables)==0 and len(row.extra_tables)==0,
    axis=1
    )
subset_performance_df["subset_table_count"] = subset_performance_df.apply(lambda row: len(row.correct_tables) + len(row.extra_tables), axis=1)
subset_performance_df["subset_column_count"] = subset_performance_df.apply(lambda row: len(row.correct_columns) + len(row.extra_columns), axis=1)

In [ ]:
from NlSqlBenchmark.NlSqlBenchmarkFactory import NlSqlBenchmarkFactory
from SchemaSubsetter.Perfect.PerfectSchemaSubsetter import PerfectSchemaSubsetter
from SubsetEvaluator.SchemaSubsetEvaluator import SchemaSubsetEvaluator, SubsetEvaluation
import os
from tqdm import tqdm
current_directory = os.getcwd()
os.chdir(current_directory.replace("/src", "/"))
fact = NlSqlBenchmarkFactory()
for bm_name in ["snails", "bird", "spider2"]:
    bm = fact.build_benchmark(bm_name)
    ss = PerfectSchemaSubsetter(fact.build_benchmark(bm_name))
    evaluator = SchemaSubsetEvaluator(use_result_cache=False)
    bm_data = {
        "database": [],
        "question_number": [],
        "total_precision": [],
        "total_f1": [],
        "table_recall": [],
        "table_precision": [],
        "table_f1": [],
        "column_recall": [],
        "column_f1": [],
        "subset_table_proportion": [],
        "subset_column_proportion": [],
        "subsetting_method": [],
        "benchmark": [],
        "comments": [],
        "eval_model": [],
        "perfect_recall": [],
        "empty_subset": [],
        "subset_table_count": [],
        "subset_column_count": []
    }
    for question in tqdm(bm):
        subset = ss.get_schema_subset(question, load_from_cache=False)
        if len(subset.schema_subset.tables) == 0:
            print(question.query)
        bm_data["database"].append(question.schema.database)
        bm_data["question_number"].append(question.question_number)
        evaluation = evaluator.evaluate_schema_subset(subset, question)
        bm_data["total_precision"] = evaluation.total_precision

        
os.chdir(current_directory)

  0%|          | 0/503 [00:00<?, ?it/s]

  2%|▏         | 10/503 [00:03<02:48,  2.92it/s]


KeyboardInterrupt: 

## Process Schema Stat Data

In [ ]:
schema_sizes = {
    # 100: "s",
    1000: "s-m",
    100000: "l+",
    # 50000: "xl",
    # 100000: "xxl"
}

def get_size(x: int) -> str:
    for s in schema_sizes.keys():
        if x < s:
            return schema_sizes[s]

schema_size_map = {
    schema_sizes[k] : k for k in schema_sizes.keys()
}

schema_stat_df["size_cat"] = schema_stat_df.column_count.apply(
    lambda x: get_size(x) 
    )
schema_stat_df["size_descr"] = schema_stat_df.size_cat.apply(
    lambda x: f"col<{schema_size_map[x]:,}"
)
schema_stat_df["size_sort"] = schema_stat_df.size_cat.apply(
    lambda x: schema_size_map[x]
)

schema_stat_df["mean_table_size"] = schema_stat_df.apply(
    lambda row: row.column_count / row.table_count,
    axis=1
)

# Failed NL to SQL

In [ ]:
join_index = ["subsetting_method", "benchmark", "database", "question_number"]
temp_data_df = nlsql_performance_df.set_index(join_index).join(
    subset_performance_df.set_index(join_index),
    lsuffix="_l", rsuffix="_r"
).reset_index()
temp_data_df["generated_query"] = temp_data_df.generated_query.fillna("no query")
temp_data_df.query(
    "eval_model_l and (generated_query=='Token limited exceeded' or generated_query=='no query') and empty_subset==False"
    )[["nl_sql_model", "subsetting_method", "database", "gold_query"]].groupby([
    "nl_sql_model", "subsetting_method", "database"
]).count()


# Execution Accuracy for Each Subsetting Method

In [ ]:
g = sns.catplot(
    data=nlsql_performance_df.query("eval_model and has_generated_query").set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).sort_values(by=["method_order", "size_sort"]),
    x="nl_sql_model",
    y="result_set_match",
    kind="bar",
    hue="subsetting_method",
    aspect=5,
    # col="size_cat",
    # col_wrap=1,
    legend_out=False,
    # legend=False
)
plt.legend(bbox_to_anchor=(0.48, -0.26), loc='lower center', borderaxespad=0, ncols=11, fontsize=16)
g.set_axis_labels("NL SQL LLM", "Execution Accuracy", fontsize=18)
g.set_xticklabels(fontsize=16)
g.set_axis_labels("NL SQL LLM", "Execution Accuracy")
g.figure.savefig("../nl_sql_results/figures/nlsql_execution_accuracy_barchart.pdf", bbox_inches="tight", dpi=300)

## Execution accuracy by benchmark, LLM, and subsetting method

In [ ]:
nlsql_performance_df.nl_sql_model.unique()

In [ ]:
g = sns.catplot(
    data=nlsql_performance_df.query("eval_model and has_generated_query").sort_values(by="method_order"),
    x="nl_sql_model",
    y="result_set_match",
    kind="bar",
    hue="subsetting_method",
    aspect=3,
    col="benchmark",
    col_wrap=1,
    legend_out=False,
    # legend=False
)
plt.legend(bbox_to_anchor=(0.45, -0.26), loc='lower center', borderaxespad=0, ncols=11)




## Execution Accuracy by Benchmark, LLM, and Database Size 

In [ ]:
g = sns.catplot(
    data=nlsql_performance_df.query("eval_model and has_generated_query").set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).sort_values(by=["method_order", "size_sort"]),
    x="nl_sql_model",
    y="result_set_match",
    kind="bar",
    hue="subsetting_method",
    aspect=5,
    col="size_cat",
    col_wrap=1,
    legend_out=True,
    # legend=False
)

plt.legend(bbox_to_anchor=(0.48, -0.26), loc='lower center', borderaxespad=0, ncols=11, fontsize=16)
g._legend.remove()
g.set_axis_labels("NL SQL LLM", "Execution Accuracy", fontsize=18)
for ax in g.axes.flat:    
    ax.set_title(ax.get_title().replace("size_cat", "Database Size"), fontsize=18)
g.figure.savefig("../nl_sql_results/figures/nlsql_execution_accuracy_barchart_sizes.pdf", bbox_inches="tight", dpi=300)


In [ ]:
g.axes_dict["l+"].set_xlabel("hi")

# Subsetting - NlSQL Performance Correlations

In [ ]:
temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="l_", rsuffix="r_"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index()

## Recall x Execution Accuracy

In [ ]:
temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index().query("eval_model_l")
sns.barplot(
    data=temp_data_df,
    x=temp_data_df.total_recall.round(1),
    y="result_set_match"
)

## Precision x Execution Accuracy

In [ ]:
temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index().query("eval_model_l and has_generated_query")
sns.barplot(
    data=temp_data_df,
    x=temp_data_df.total_precision.round(1),
    y="result_set_match"
)

## Table proportion x Execution Accuracy

In [ ]:
temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index()
sns.barplot(
    data=temp_data_df.query("subset_table_proportion <= 1"),
    x=temp_data_df.query("subset_table_proportion <= 1 and eval_model_l and has_generated_query").subset_table_proportion.round(1),
    y="result_set_match"
)

## Table count x Execution Accuracy

In [ ]:
temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index()
sns.barplot(
    data=temp_data_df.query("eval_model_l and has_generated_query"),
    x="subset_table_count",
    y="result_set_match"
)

## Logistic Regression

### All subsetting metrics predict execution accuracy

In [ ]:
import statsmodels.api as sm

temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index().query("eval_model_l and has_generated_query").dropna()
temp_data_df['result_set_match_numeric'] = temp_data_df['result_set_match'].astype(int)
temp_data_df["schema_size_numeric"] = temp_data_df.size_cat.apply(lambda x: {
    "s": 1,
    "m": 2,
    "s-m": 1.5,
    "l": 3,
    "l+": 3,
    "xl": 4,
    "xxl": 5
}[x])

X = temp_data_df[[
    "perfect_recall",
    "table_recall",
    "table_precision",
    "column_recall",
    "column_precision",
    "subset_table_count",
    "subset_column_count",
    "schema_size_numeric"
    ]]
y = temp_data_df['result_set_match_numeric']
X = sm.add_constant(X)
logit_model = sm.Logit(y, X).fit()

# Print the summary of the logistic regression model
print(logit_model.summary())

### Only perfect recall

In [ ]:
import statsmodels.api as sm

temp_data_df = subset_performance_df.set_index(
    ["subsetting_method", "benchmark", "database", "question_number"]
    ).join(
        nlsql_performance_df.set_index(
        ["subsetting_method", "benchmark", "database", "question_number"]
        ),
        lsuffix="_l", rsuffix="_r"
    ).reset_index().set_index(["benchmark", "database"]).join(
        schema_stat_df.set_index(["benchmark", "database"])
    ).reset_index().query("perfect_recall == 1 and eval_model_l and has_generated_query").dropna()
temp_data_df['result_set_match_numeric'] = temp_data_df['result_set_match'].astype(int)
temp_data_df["schema_size_numeric"] = temp_data_df.size_cat.apply(lambda x: {
    "s": 1,
    "s-m": 1.5,
    "m": 2,
    "l": 3,
    "l+": 3,
    "xl": 4,
    "xxl": 5
}[x])

X = temp_data_df[[
    "subset_table_count", 
    "schema_size_numeric"
    ]]
y = temp_data_df['result_set_match_numeric']
X = sm.add_constant(X)
logit_model = sm.Logit(y, X).fit()

# Print the summary of the logistic regression model
print(logit_model.summary())